In [1]:
# import API KEY

import os                           # operating system library
from dotenv import load_dotenv      # load environment variables  


load_dotenv()


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
from langchain_openai.chat_models import ChatOpenAI   # LangChain connection to OpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4-turbo")

response = model.invoke("What is the Suez Canal?")

response.content

"The Suez Canal is a man-made waterway in Egypt, connecting the Mediterranean Sea to the Red Sea. This canal is a crucial international shipping route as it allows ships to travel between Europe and South Asia without navigating around Africa, thereby significantly reducing travel time and distance. Constructed by the French engineer Ferdinand de Lesseps, the canal was completed in 1869 after ten years of work.\n\nThe Suez Canal is significant for global trade; it facilitates the easy and quick transport of goods between continents. The canal operates as a sea-level waterway without locks, and its operation is managed by the Suez Canal Authority of Egypt. It has been expanded and modernized several times to accommodate larger ships, notably with a major expansion project completed in 2015 that allowed two-way traffic along part of the route to increase the canal's capacity.\n\nDue to its strategic importance, the Suez Canal has also been a focal point in geopolitical conflicts. It was 

In [3]:
os.listdir("../pdfs")

['.DS_Store', '2024_BRIEFING.pdf']

In [4]:
from langchain_community.document_loaders import PyPDFDirectoryLoader

In [5]:
loader = PyPDFDirectoryLoader("../pdfs/")

pages = loader.load()

In [6]:
len(pages)

8

In [7]:
from langchain_openai.embeddings import OpenAIEmbeddings


vectorizer = OpenAIEmbeddings()

In [8]:
from langchain_community.vectorstores import Chroma

chroma_db = Chroma.from_documents(pages, vectorizer, persist_directory="../chroma_db")

In [9]:
retriever = chroma_db.as_retriever(search_type="mmr", search_kwargs={"k": 2, "lambda_mult": 0.25})

In [10]:
from langchain.prompts import ChatPromptTemplate

In [11]:
template = """
            Given the context below and the question, 
            please generate a header and 5 bullet points.
            List with numbers the bullet points.
            Summarize each bullet point in 40 words.
            Also fetch numbers to describe the information.

            Context: {context}

            Question: {question}
            """


prompt = ChatPromptTemplate.from_template(template)

In [12]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [13]:
from langchain_core.runnables import RunnablePassthrough

In [14]:
query = "What are the endnotes of the briefing?"

In [15]:
in_chain = {"context": retriever, "question": RunnablePassthrough()} | prompt | model | parser


response = in_chain.invoke(query)

In [16]:
response.split('\n')

['### Endnotes of the March 2024 EPRS Briefing on Red Sea Threats',
 '',
 '1. **Shipping Rate Increases**',
 '   - Freight rates from Shanghai to Europe escalated significantly, with a reported increase of 256%. In contrast, routes to the US west coast saw a lesser increase of 162%.',
 '',
 '2. **Rising Insurance Costs**',
 '   - Insurance rates for vessels with cargo valued at $100 million surged from 0.1% to 0.5% or more, translating to an additional cost ranging from $0.5 million to $1 million.',
 '',
 '3. **Market Index Fluctuations**',
 "   - The Baltic Dry Index peaked at $3,000 in mid-December 2023 but dropped to $1,503 by January. Similarly, Drewry's World Container Index spiked from $1,521 to $3,777 within the same timeframe.",
 '',
 '4. **Prevention of Environmental Disaster**',
 '   - A potential environmental crisis was averted in August 2023 when the contents of the oil tanker FSO Safer were safely removed, preventing a major oil spill.',
 '',
 '5. **Impact on Manufacturin

In [17]:
from langchain_openai import OpenAI

input_model = OpenAI(temperature=0, max_tokens=1024)

In [18]:
template = """
            We have provided  information below.
            Given this information, please generate python-pptx code for a single 
            slide with this information set header as title and  
            bullet points.
            
            Put the title as the first cell of the slide.
            
            Separate the bullet points into separate texts with line separator.
            
            Set font size to minimum for fixing text to cells. 
            Save the file in ../pptx/Red Sea Security Threats.pptx path

            Information: {context}
            """


prompt = ChatPromptTemplate.from_template(template)

In [19]:
out_chain = prompt | input_model | parser

In [ ]:
output = out_chain.invoke({"context": response})

In [ ]:
output.split('\n')

In [ ]:
exec('print(2+2)')

In [ ]:
exec(output)